# CSV DFJ and Regional Level

## Contents
#### Setup
1. [import_packages](#import_packages) 

#### Stage 1 - [CSV DFJ and Regional Level](#CSV_DFJ_and_Regional_Level)
2. [divorce_dfj_table](#divorce_dfj_table) -  imports Divorce CSV dfj data from S3 bucket into a temporary table
3. [fr_dfj_table](#fr_dfj_table) -  imports Financial Remedy CSV dfj data from S3 bucket into a temporary table
4. [care_dfj_table](#care_dfj_table) -  imports Care and Supervision CSV dfj data from S3 bucket into a temporary table
5. [dfj_temp](#dfj_temp) - joins all the CSV dfj outputs from each case type (Adoption, Care and Supervision, Domestic Violence, Divorce, Financial Remedy, Private Law, and Public Law)

## 1. Import packages and set options 
<a name="import_packages"></a>

In [ ]:
import pandas as pd  # a module which provides the data structures and functions to store and manipulate tables in dataframes
import pydbtools as pydb  # A module which allows SQL queries to be run on the Analytical Platform from Python, see https://github.com/moj-analytical-services/pydbtools
import boto3  # allows you to directly create, update, and delete AWS resources from Python scripts
import numpy as np
import re

# sets parameters to view dataframes for tables easier
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

## Stage 1 - CSV DFJ and Regional Level
<a name="CSV_DFJ_and_Regional_Level"></a>

### Import Divorce DFJ
<a name="divorce_dfj_table"></a>

#### Create the divorce_dfj table

In [ ]:
#imports Divorce DFJ data from S3 bucket into a temporary table
divorce_dfj_table = pd.read_csv("s3://alpha-family-data/CSVs/DFJ/Divorce_DFJ_CSV.csv", low_memory=False)

In [ ]:
pydb.dataframe_to_temp_table(divorce_dfj_table, "divorce_dfj")

##### divorce_dfj validation

In [ ]:
#divorce_dfj_count = pydb.read_sql_query("SELECT * from __temp__.divorce_dfj limit 10")
#divorce_dfj_count

### Import Financial Remedy DFJ
<a name="fr_dfj_table"></a>

#### Create the fr_dfj table

In [ ]:
#imports Financial Remedy DFJ data from S3 bucket into a temporary table
fr_dfj_table = pd.read_csv("s3://alpha-family-data/CSVs/DFJ/FR_DFJ.csv", low_memory=False)

In [ ]:
pydb.dataframe_to_temp_table(fr_dfj_table, "fr_dfj")

##### fr_dfj validation

In [ ]:
#fr_dfj_count = pydb.read_sql_query("SELECT * from __temp__.fr_dfj limit 10")
#fr_dfj_count

### Import Care and Supervision DFJ
<a name="care_dfj_table"></a>

#### *Temporary - once Public law amalgamation is complete and Children Act on the AP includes Public law timeliness - this code can be removed*

#### Create the care_dfj table

In [ ]:
#imports Care and Supervision DFJ data from S3 bucket into a temporary table
care_dfj_table = pd.read_csv("s3://alpha-family-data/CSVs/DFJ/care_and_supervision_dfj.csv", low_memory=False)

In [ ]:
pydb.dataframe_to_temp_table(care_dfj_table, "care_dfj")

##### care_dfj validation

In [ ]:
#care_dfj_count = pydb.read_sql_query("SELECT * from __temp__.care_dfj limit 10")
#care_dfj_count

### Final Output CSV Timeliness
<a name="dfj_temp"></a>

In [ ]:
pydb.create_temp_table(
f""" 
SELECT * FROM fcsq.adopt_dfj

UNION

SELECT * FROM fcsq.ca_dfj

UNION

SELECT * FROM fcsq.domestic_violence_dfj

UNION

SELECT * FROM __temp__.fr_dfj

UNION

SELECT * FROM __temp__.care_dfj

UNION

SELECT *,
NULL as cases
FROM __temp__.divorce_dfj

""",

"dfj_temp")

In [ ]:
#dfj_temp = pydb.read_sql_query("SELECT * FROM __temp__.dfj_temp;")
#dfj_temp

In [ ]:
#Orders the data by year, country, and county_ua
dfj = pydb.read_sql_query("""
SELECT *
from __temp__.dfj_temp
ORDER BY category,
stage,
year,
quarter,
dfj_new,
region
""")

In [ ]:
#Export the final csv
dfj.to_csv("s3://alpha-family-data/CSVs/DFJ/CSV DFJ and Regional Level.csv", index = False)